In [33]:
import sys
sys.path.insert(0, '..')

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from os import listdir
from os.path import join
import deepcoloring as dc
%matplotlib inline
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In order to run this example you need to download dataset https://www.plant-phenotyping.org/datasets-download

In [2]:
basepath = "/media/hpc-4_Raid/vkulikov/CVPPP2017_LSC_training/training/A1/"
rgb = sorted([join(basepath,f) for f in listdir(basepath) if f.endswith('_rgb.png')])
labels = sorted([join(basepath,f) for f in listdir(basepath) if f.endswith('_label.png')])

reader = dc.Reader(rgb,labels,2)

In [31]:
transforms = [dc.rgba2rgb(),
              dc.clip_patch((192,192)),
        dc.flip_horizontally(),
        dc.flip_vertically(),
        dc.utils.rotate90(),
        dc.normalize(0.5,0.5)]

generetor = reader.create_batch_generator(10,transforms=transforms)
mask_builder = dc.build_halo_mask()

In [41]:
net = dc.EUnet(3,6,2,2,2,padding=1,init_xavier=True,use_bn=True,use_dropout=True).to(device)

optim = torch.optim.Adam(net.parameters(),lr=0.001)
for i in range(10):
    x,y = generetor()
    x_t = torch.from_numpy(x).to(device)
    optim.zero_grad()
    y_pred = net(x_t)
    labels,halo,objects = mask_builder(y)
    loss = dc.halo_loss(y_pred,labels,halo,objects)
    loss.backward()
    optim.step()

tensor([ 0.9069], device='cuda:0')
tensor([ 0.7925], device='cuda:0')
tensor([ 0.7190], device='cuda:0')
tensor([ 0.6925], device='cuda:0')
tensor([ 0.6518], device='cuda:0')
tensor([ 0.6315], device='cuda:0')
tensor([ 0.5354], device='cuda:0')
tensor([ 0.5774], device='cuda:0')
tensor([ 0.6140], device='cuda:0')
tensor([ 0.6272], device='cuda:0')
